In [ ]:
import rasterio
import os
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from rasterio.plot import show
import rasterio
import numpy as np
import requests

In [ ]:
# # update the wd path to be able to laod the modules
# # os.getcwd()
os.chdir('..')

In [ ]:
os.environ['GCS_BUCKET']='gee-exports'
os.environ['GOOGLE_APPLICATION_USER']='developers@citiesindicators.iam.gservaiceaccount.com'
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='/Users/jt/dev/wri-notebooks/server/citiesindicators-credentials.json'

In [ ]:
from city_metrix import (
    built_land_without_tree_cover, 
    mean_tree_cover, 
    built_land_with_low_surface_reflectivity, 
    built_land_with_high_land_surface_temperature
)


# Select a city

In [5]:
# Get city boundary
cities_api_url = "http://3.225.116.209:8000/cities"
city_name = "BRA-Salvador"
admin_level = "ADM4union"
city_geom = cities_api_url + "/"+city_name+"/"+admin_level+"/geojson"
city_gdf = gpd.read_file(city_geom)
city_gdf.head()


,id,geo_id,geo_name,geo_level,geo_parent_name,geo_version,geometry
0,0,BRA-Salvador_ADM4-union_1,BRA-Salvador,ADM4union,BRA-Salvador,0,"MULTIPOLYGON (((-38.50135 -13.01134, -38.50140..."


# Calculate indicators - AOI level

## Compute indicators: One city * One indicator

### HEA-3 Built land with Low Surface reflectivity

In [6]:
%%time

city_gdf["built_land_with_low_surface_reflectivity"] = built_land_with_low_surface_reflectivity(city_gdf) 
city_gdf

Extracting ESA world cover layer:
[########################################] | 100% Completed | 5.74 ss
Calculating albedo layer:
[########################################] | 100% Completed | 107.24 s
Extracting ESA world cover layer:
[########################################] | 100% Completed | 5.85 ss
CPU times: user 19.5 s, sys: 3.36 s, total: 22.8 s
Wall time: 2min 8s


,id,geo_id,geo_name,geo_level,geo_parent_name,geo_version,geometry,built_land_with_low_surface_reflectivity
0,0,BRA-Salvador_ADM4-union_1,BRA-Salvador,ADM4union,BRA-Salvador,0,"MULTIPOLYGON (((-38.50135 -13.01134, -38.50140...",0.786551


### LND-2 Land with Tree cover

In [6]:
%%time

city_gdf["mean_tree_cover"] = mean_tree_cover(city_gdf) 
city_gdf

CPU times: user 6.03 s, sys: 1.04 s, total: 7.07 s
Wall time: 11.6 s


,id,geo_id,geo_name,geo_level,geo_parent_name,geo_version,geometry,built_land_with_low_surface_reflectivity,mean_tree_cover
0,0,BRA-Salvador_ADM4-union_1,BRA-Salvador,ADM4union,BRA-Salvador,0,"MULTIPOLYGON (((-38.50135 -13.01134, -38.50140...",0.786551,28.366773


In [21]:
import ee
import geemap

WC = ee.ImageCollection("ESA/WorldCover/v100")
WorldCover = WC.first();
builtup = WorldCover.eq(50)

## read Trees in Mosaic Landscapes tree cover dataset
TML = ee.ImageCollection('projects/wri-datalab/TML')
TreeCoverImg = TML.reduce(ee.Reducer.mean()).rename('b1')

# https://gis.stackexchange.com/questions/421422/google-earth-engine-image-no-data-mask
TreeDataMask = TreeCoverImg.unmask(-99).neq(-99)   # give no data pixels an abitrary value outside the range of the data. Then make a boolean raster using the assigned value.

fc = geemap.geopandas_to_ee(city_gdf)
pixelcounts = TreeCoverImg.reduceRegions(fc,ee.Reducer.mean(),10)
df = geemap.ee_to_pandas(pixelcounts)
df

,built_land_with_low_surface_reflectivity,geo_id,geo_level,geo_name,geo_parent_name,geo_version,id,mean
0,0.240654,BRA-Salvador_ADM4-union_1,ADM4union,BRA-Salvador,BRA-Salvador,0,0,28.349677


### HEA-4 Percent of Built land without tree cover

In [7]:
%%time

city_gdf["built_land_without_tree_cover"] = built_land_without_tree_cover(city_gdf) 
city_gdf

CPU times: user 16.2 s, sys: 2.7 s, total: 18.9 s
Wall time: 2min 53s


,id,geo_id,geo_name,geo_level,geo_parent_name,geo_version,geometry,built_land_with_low_surface_reflectivity,mean_tree_cover,built_land_without_tree_cover
0,0,BRA-Salvador_ADM4-union_1,BRA-Salvador,ADM4union,BRA-Salvador,0,"MULTIPOLYGON (((-38.50135 -13.01134, -38.50140...",0.786551,28.366773,0.907382


### HEA-2 Built Land With High LST

In [7]:
%%time

city_gdf["built_land_with_high_land_surface_temperature"] = built_land_with_high_land_surface_temperature(city_gdf) 
city_gdf

Extracting ESA world cover layer:
[########################################] | 100% Completed | 5.82 ss
Calculating land surface temperature layer:
[########################################] | 100% Completed | 1.74 sms
Extracting ESA world cover layer:
[########################################] | 100% Completed | 5.71 ss
CPU times: user 10.8 s, sys: 1.64 s, total: 12.4 s
Wall time: 18.2 s


,id,geo_id,geo_name,geo_level,geo_parent_name,geo_version,geometry,built_land_with_low_surface_reflectivity,built_land_with_high_land_surface_temperature
0,0,BRA-Salvador_ADM4-union_1,BRA-Salvador,ADM4union,BRA-Salvador,0,"MULTIPOLYGON (((-38.50135 -13.01134, -38.50140...",0.786551,0.096108


## Compute indicators: One city * Multiple indicators

In [40]:
%%time

indicators =get_indicators(city_gdf, indicators=[Indicator.TREE_COVER,
                                                 Indicator.BUILT_LAND_WITH_TREE_COVER])
indicators_df = pd.merge(indicators[0], indicators[1])
indicators_df

CPU times: total: 3.31 s
Wall time: 8.63 s


,id,geo_id,geo_name,geo_level,geo_parent_name,geo_version,geometry,LND_2_percentTreeCover,HEA_4_percentBuiltupWithoutTreeCover
0,0,BRA-Salvador_ADM4-union_1,BRA-Salvador,ADM4union,BRA-Salvador,0,"MULTIPOLYGON (((-38.50135 -13.01134, -38.50140...",0.283478,0.907352


# Caculate indicators - Subcity levels

# Get layers

## Admin Boundary

## Esa World Cover

## Albedo

## Trpical Tree cover

## LST